In [ ]:
import sys
import keras
import pygame
import numpy as np
from PIL import Image

In [ ]:
model = keras.models.load_model('digit_recognition_model.h5')

def recognize_number(screen):
    raw_str = pygame.image.tostring(screen, 'RGBA', False)
    image = Image.frombytes('RGBA', (width, height), raw_str)
    image = image.convert('L')
    segments = segment_image(image)
    number = ''
    for segment in segments:
        segment = segment.resize((28, 28), Image.Resampling.LANCZOS)
        segment_array = np.array(segment).reshape(1, 28, 28, 1) / 255.0
        prediction = model.predict(segment_array)
        predicted_digit = np.argmax(prediction)
        if predicted_digit == 10:
            predicted_digit = '-'
        number += str(predicted_digit)
    print(f"Распознанное число: {number}")
    return number


def segment_image(image):
    segment_width = 196
    segments = []
    for i in range(5):
        left = i * segment_width
        right = left + segment_width
        segment = image.crop((left, 0, right, image.height))
        segments.append(segment)
    return segments

In [ ]:
pygame.init()

width, height = 980, 196
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Рисование чисел")


def clear_screen():
    screen.fill((0, 0, 0))
    for x in range(0, width, height):
        pygame.draw.line(screen, (128, 128, 128), (x, 0), (x, height))

drawing = False
last_pos = None
color = (255, 255, 255)
radius = 4
clear_screen()

def draw_line(screen, start_pos, end_pos, color, radius):
    x1, y1 = start_pos
    x2, y2 = end_pos
    dx = x2 - x1
    dy = y2 - y1
    distance = max(abs(dx), abs(dy))
    for i in range(distance):
        x = int(x1 + float(i) / distance * dx)
        y = int(y1 + float(i) / distance * dy)
        pygame.draw.circle(screen, color, (x, y), radius)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEBUTTONDOWN:
            drawing = True
            last_pos = event.pos

        if event.type == pygame.MOUSEBUTTONUP:
            drawing = False

        if event.type == pygame.MOUSEMOTION:
            if drawing:
                mouse_pos = event.pos
                draw_line(screen, last_pos, mouse_pos, color, radius)
                last_pos = mouse_pos

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                number = recognize_number(screen)
                pygame.display.set_caption(number)
            elif event.key == pygame.K_BACKSPACE:
                clear_screen()

    pygame.display.flip()
